In [3]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value

# 問題設定
problem = LpProblem("mTSP_with_Node_Constraints", LpMinimize)

# ノードとエッジの設定
nodes = [1, 2, 3, 4]  # ノード番号
# 同じノード間の移動を除外し、エッジコストを定義
edges = {(i, j): 1 for i in nodes for j in nodes if i != j}

salesmen = [1, 2]  # セールスマンの人数

# 変数の定義
x = LpVariable.dicts('x', [(i, j, k) for i in nodes for j in nodes for k in salesmen], 0, 1, LpBinary)

# 目的関数: 総コストの最小化
problem += lpSum(edges[i, j] * x[i, j, k] for i in nodes for j in nodes if i != j for k in salesmen)


# 制約: 各ノードは1回だけ訪問される
for j in nodes:
    problem += lpSum(x[i, j, k] for i in nodes if i != j for k in salesmen) == 1

# 始点と終点が同じ
for k in salesmen:
    problem += lpSum(x[1, j, k] for j in nodes if j != 1) == 1  # 始点から1度だけ出発
    problem += lpSum(x[i, 1, k] for i in nodes if i != 1) == 1  # 終点に1度だけ戻る

# 特定のノードに関する制約
# 例: セールスマン1はノード2を通過しなければならない
problem += lpSum(x[2, j, 1] for j in nodes if j != 2) >= 1
problem += lpSum(x[i, 2, 1] for i in nodes if i != 2) >= 1

# 例: セールスマン2はノード3を通過してはならない
problem += lpSum(x[3, j, 2] for j in nodes if j != 3) == 0
problem += lpSum(x[i, 3, 2] for i in nodes if i != 3) == 0

# 問題を解く
problem.solve()

# 結果の出力
print(f"Status: {LpStatus[problem.status]}")
print(f"Objective value: {value(problem.objective)}")

# 各セールスマンのルート出力
for k in salesmen:
    print(f"Salesman {k} route:")
    for i in nodes:
        for j in nodes:
            if i != j and value(x[i, j, k]) == 1:
                print(f"{i} -> {j}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kanaoka/usr/github/caredx2024_rapid_prototyping/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e97a74f695c14632bf4632dfc6b369ee-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/e97a74f695c14632bf4632dfc6b369ee-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 138 RHS
At line 151 BOUNDS
At line 176 ENDATA
Problem MODEL has 12 rows, 24 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Infeasible
Objective value: 5.0
Salesman 1 route:
1 -> 4
2 -> 1
3 -> 2
4 -> 3
Salesman 2 route:
1 -> 4
2 -> 1


In [10]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value
import random 

# シード固定
random.seed(0)

# 問題設定
problem = LpProblem("mTSP_with_Node_Constraints", LpMinimize)

# ノードとエッジの設定
nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9]  # ノード番号
edges = {(i, j): random.random() for i in nodes for j in nodes if i != j}  # コストは1で固定

salesmen = [1, 2]  # セールスマンの人数

# 変数の定義
x = LpVariable.dicts('x', [(i, j, k) for i in nodes for j in nodes for k in salesmen], 0, 1, LpBinary)

# 各セールスマンのコストを表す変数
costs = LpVariable.dicts('cost', salesmen, lowBound=0, cat='Continuous')

# 全セールスマンの最大コストと最小コストを表す変数
max_cost = LpVariable('max_cost', lowBound=0, cat='Continuous')
min_cost = LpVariable('min_cost', lowBound=0, cat='Continuous')

# 各セールスマンのコストを計算
for k in salesmen:
    problem += costs[k] == lpSum(edges[i, j] * x[i, j, k] for i in nodes for j in nodes if i != j)

# 最大コストと最小コストを制約により定義
problem += max_cost >= lpSum(costs[k] for k in salesmen)
problem += min_cost <= lpSum(costs[k] for k in salesmen)

# 目的関数: 最大コストと最小コストの差を最小化
problem += max_cost - min_cost

# 制約: 各ノードは1回だけ訪問される
for j in nodes:
    problem += lpSum(x[i, j, k] for i in nodes if i != j for k in salesmen) == 1

# 始点と終点が同じ
for k in salesmen:
    problem += lpSum(x[1, j, k] for j in nodes if j != 1) == 1  # 始点から1度だけ出発
    problem += lpSum(x[i, 1, k] for i in nodes if i != 1) == 1  # 終点に1度だけ戻る

# 特定のノードに関する制約
# 例: セールスマン1はノード2を通過しなければならない
problem += lpSum(x[2, j, 1] for j in nodes if j != 2) >= 1
problem += lpSum(x[i, 2, 1] for i in nodes if i != 2) >= 1

# 例: セールスマン2はノード3を通過してはならない
problem += lpSum(x[3, j, 2] for j in nodes if j != 3) == 0
problem += lpSum(x[i, 3, 2] for i in nodes if i != 3) == 0

# 問題を解く
problem.solve()

# 結果の出力
print(f"Status: {LpStatus[problem.status]}")
print(f"Objective value (Max Cost - Min Cost): {value(problem.objective)}")

# 各セールスマンのコスト出力
for k in salesmen:
    print(f"Salesman {k} cost: {value(costs[k])}")

# 各セールスマンのルート出力
for k in salesmen:
    print(f"Salesman {k} route:")
    for i in nodes:
        for j in nodes:
            if i != j and value(x[i, j, k]) == 1:
                print(f"{i} -> {j}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kanaoka/usr/github/caredx2024_rapid_prototyping/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5f488c3df2914dd7b70adc3cb813b932-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5f488c3df2914dd7b70adc3cb813b932-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 26 COLUMNS
At line 677 RHS
At line 699 BOUNDS
At line 844 ENDATA
Problem MODEL has 21 rows, 148 columns and 360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Infeasible
Objective value (Max Cost - Min Cost): 0.0
Salesman 1 cost: 1.4572289
Salesman 2 cost: 0.31735443
Salesman 1 route:
1 -> 5
2 -> 6
4 -> 2
4 -> 3
6 -> 1
6 -> 8
9 -> 7
Salesman 2 ro

In [6]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value

# 問題設定
problem = LpProblem("mTSP_with_Node_Constraints", LpMinimize)

# ノードとエッジの設定
nodes = [1, 2, 3, 4]  # ノード番号
edges = {(i, j): 1 for i in nodes for j in nodes if i != j}  # コストは1で固定

salesmen = [1, 2]  # セールスマンの人数

# 変数の定義
x = LpVariable.dicts('x', [(i, j, k) for i in nodes for j in nodes for k in salesmen], 0, 1, LpBinary)

# フロー変数（サブツアー除去制約のための補助変数）
u = LpVariable.dicts('u', [(i, k) for i in nodes for k in salesmen], lowBound=0, cat='Continuous')

# 各セールスマンのコストを表す変数
costs = LpVariable.dicts('cost', salesmen, lowBound=0, cat='Continuous')

# 全セールスマンの最大コストと最小コストを表す変数
max_cost = LpVariable('max_cost', lowBound=0, cat='Continuous')
min_cost = LpVariable('min_cost', lowBound=0, cat='Continuous')

# 各セールスマンのコストを計算
for k in salesmen:
    problem += costs[k] == lpSum(edges[i, j] * x[i, j, k] for i in nodes for j in nodes if i != j)

# 最大コストと最小コストを制約により定義
problem += max_cost >= lpSum(costs[k] for k in salesmen)
problem += min_cost <= lpSum(costs[k] for k in salesmen)

# 目的関数: 最大コストと最小コストの差を最小化
problem += max_cost - min_cost

# 制約: 各ノードは1回だけ訪問される
for j in nodes:
    problem += lpSum(x[i, j, k] for i in nodes if i != j for k in salesmen) == 1

# 始点をノード1に固定
for k in salesmen:
    # ノード1から出発するエッジは1つだけ
    problem += lpSum(x[1, j, k] for j in nodes if j != 1) == 1
    # セールスマンはノード1に戻る
    problem += lpSum(x[i, 1, k] for i in nodes if i != 1) == 1

# サブツアー除去制約（MTZ制約）
for k in salesmen:
    for i in nodes:
        for j in nodes:
            if i != j and i != 1 and j != 1:
                problem += u[i, k] - u[j, k] + (len(nodes) * x[i, j, k]) <= len(nodes) - 1

# 特定のノードに関する制約
# 例: セールスマン1はノード2を通過しなければならない
problem += lpSum(x[2, j, 1] for j in nodes if j != 2) >= 1
problem += lpSum(x[i, 2, 1] for i in nodes if i != 2) >= 1

# 例: セールスマン2はノード3を通過してはならない
problem += lpSum(x[3, j, 2] for j in nodes if j != 3) == 0
problem += lpSum(x[i, 3, 2] for i in nodes if i != 3) == 0

# 問題を解く
problem.solve()

# 結果の出力
print(f"Status: {LpStatus[problem.status]}")
print(f"Objective value (Max Cost - Min Cost): {value(problem.objective)}")

# 各セールスマンのコスト出力
for k in salesmen:
    print(f"Salesman {k} cost: {value(costs[k])}")

# 各セールスマンのルート出力
for k in salesmen:
    print(f"Salesman {k} route:")
    for i in nodes:
        for j in nodes:
            if i != j and value(x[i, j, k]) == 1:
                print(f"{i} -> {j}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kanaoka/usr/github/caredx2024_rapid_prototyping/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1bffbf08e7204283bcbefcc73bc012bf-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/1bffbf08e7204283bcbefcc73bc012bf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 33 COLUMNS
At line 200 RHS
At line 229 BOUNDS
At line 254 ENDATA
Problem MODEL has 28 rows, 34 columns and 116 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Infeasible
Objective value (Max Cost - Min Cost): 0.0
Salesman 1 cost: 3.0
Salesman 2 cost: 2.0
Salesman 1 route:
1 -> 2
1 -> 3
2 -> 1
Salesman 2 route:
1 -> 4
2 -> 1


In [12]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value
import random

# 問題設定
problem = LpProblem("mTSP_with_Node_Constraints", LpMinimize)

# ノードとエッジの設定
nodes = [1, 2, 3, 4, 5]  # ノード番号
edges = {(i, j): random.random() for i in nodes for j in nodes if i != j}  # コストは1で固定
salesmen = [1, 2]  # セールスマンの人数

# 開始ノードの設定
start_node = 1

# 変数の定義
x = LpVariable.dicts('x', [(i, j, k) for i in nodes for j in nodes for k in salesmen], 0, 1, LpBinary)

# フロー変数（サブツアー除去制約のための補助変数）
u = LpVariable.dicts('u', [(i, k) for i in nodes for k in salesmen], lowBound=0, cat='Continuous')

# 各セールスマンのコストを表す変数
costs = LpVariable.dicts('cost', salesmen, lowBound=0, cat='Continuous')

# 全セールスマンの最大コストと最小コストを表す変数
max_cost = LpVariable('max_cost', lowBound=0, cat='Continuous')
min_cost = LpVariable('min_cost', lowBound=0, cat='Continuous')

# 各セールスマンのコストを計算
for k in salesmen:
    problem += costs[k] == lpSum(edges[i, j] * x[i, j, k] for i in nodes for j in nodes if i != j)

# 最大コストと最小コストを制約により定義
problem += max_cost >= lpSum(costs[k] for k in salesmen)
problem += min_cost <= lpSum(costs[k] for k in salesmen)

# 目的関数: 最大コストと最小コストの差を最小化
problem += max_cost - min_cost

# 制約: 各ノードは1回だけ訪問される
for j in nodes:
    problem += lpSum(x[i, j, k] for i in nodes if i != j for k in salesmen) == 1

# 始点をノード1に固定
for k in salesmen:
    # ノード1から出発するエッジは1つだけ
    problem += lpSum(x[1, j, k] for j in nodes if j != 1) == 1
    # セールスマンはノード1に戻る
    problem += lpSum(x[i, 1, k] for i in nodes if i != 1) == 1

# サブツアー除去制約（MTZ制約）
for k in salesmen:
    for i in nodes:
        if i != 1:  # ノード1は含めない
            for j in nodes:
                if i != j and j != 1:
                    problem += u[i, k] - u[j, k] + (len(nodes) * x[i, j, k]) <= len(nodes) - 1

# uの範囲制約: ノード1を基準にして順番を表現
for k in salesmen:
    for i in nodes:
        if i != 1:
            problem += u[i, k] >= 1
            problem += u[i, k] <= len(nodes) - 1

# 特定のノードに関する制約
# 例: セールスマン1はノード2を通過しなければならない
problem += lpSum(x[2, j, 1] for j in nodes if j != 2) >= 1
problem += lpSum(x[i, 2, 1] for i in nodes if i != 2) >= 1

# 例: セールスマン2はノード3を通過してはならない
problem += lpSum(x[3, j, 2] for j in nodes if j != 3) == 0
problem += lpSum(x[i, 3, 2] for i in nodes if i != 3) == 0

# 問題を解く
problem.solve()

# 結果の出力
print(f"Status: {LpStatus[problem.status]}")
# print(f"Objective value (Max Cost - Min Cost): {value(problem.objective)}")

# 各セールスマンのコスト出力
for k in salesmen:
    print(f"Salesman {k} cost: {value(costs[k])}")

# 各セールスマンのルート出力
for k in salesmen:
    print(f"Salesman {k} route:")
    for i in nodes:
        for j in nodes:
            if i != j and value(x[i, j, k]) == 1:
                print(f"{i} -> {j}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kanaoka/usr/github/caredx2024_rapid_prototyping/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/813227d9cd304dafa07433df11b0d516-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/813227d9cd304dafa07433df11b0d516-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 62 COLUMNS
At line 353 RHS
At line 411 BOUNDS
At line 452 ENDATA
Problem MODEL has 57 rows, 52 columns and 208 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Infeasible
Salesman 1 cost: 0.77267987
Salesman 2 cost: 1.0400829
Salesman 1 route:
1 -> 5
2 -> 1
Salesman 2 route:
1 -> 5
3 -> 4
4 -> 3
5 -> 1


In [24]:
import pulp
import numpy as np

# ノード数
num_nodes = 5

# セールスマンの人数
num_salesmen = 2

# ノード間のコスト（距離）をランダムに設定
np.random.seed(42)
cost_matrix = np.random.random(size=(num_nodes, num_nodes))
np.fill_diagonal(cost_matrix, 0)

# コスト行列を表示
print("Cost Matrix:")
print(cost_matrix)

# PuLP問題の定義
prob = pulp.LpProblem("mTSP", pulp.LpMinimize)

# 変数の定義
x = pulp.LpVariable.dicts("x", (range(num_nodes), range(num_nodes), range(num_salesmen)), cat="Binary")
u = pulp.LpVariable.dicts("u", (range(num_nodes), range(num_salesmen)), lowBound=0, cat="Continuous")

# 始点/終端ノードを固定（例: ノード0）
start_end_node = 0

# 目的関数の設定：セールスマンの総コストを最小化
prob += pulp.lpSum(cost_matrix[i][j] * x[i][j][k] for i in range(num_nodes) for j in range(num_nodes) for k in range(num_salesmen))

# 制約条件1: 各ノードには必ず1人のセールスマンが訪れる
for j in range(num_nodes):
    prob += pulp.lpSum(x[i][j][k] for i in range(num_nodes) for k in range(num_salesmen) if i != j) == 1

# # 制約条件2: 各ノードからは必ず1つのノードに移動する
# for i in range(num_nodes):
#     prob += pulp.lpSum(x[i][j][k] for j in range(num_nodes) for k in range(num_salesmen) if i != j) == 1

# 制約条件3: サブツアーを排除するためのミラー・フロー制約
# for k in range(num_salesmen):
#     for i in range(1, num_nodes):
#         for j in range(1, num_nodes):
#             if i != j:
#                 prob += u[i][k] - u[j][k] + (num_nodes) * x[i][j][k] <= num_nodes - 1

# 制約条件4: 始点/終端ノードを固定
for k in range(num_salesmen):
    # 始点ノードから出発する
    prob += pulp.lpSum(x[start_end_node][j][k] for j in range(num_nodes) if start_end_node != j) == 1
    # 終端ノードに戻る
    prob += pulp.lpSum(x[i][start_end_node][k] for i in range(num_nodes) if start_end_node != i) == 1

# 問題を解く
prob.solve()

# 結果の表示
print("\nStatus:", pulp.LpStatus[prob.status])

# 各セールスマンの巡回経路の表示
for k in range(num_salesmen):
    print(f"\nSalesman {k+1}'s route:")
    current_node = start_end_node
    route = [current_node]
    while True:
        next_node = None
        for j in range(num_nodes):
            if current_node != j and pulp.value(x[current_node][j][k]) == 1:
                next_node = j
                route.append(j)
                current_node = j
                break
        if next_node == start_end_node or next_node is None:
            break
    print(route)

# 総コストの表示
print("\nTotal Cost:", pulp.value(prob.objective))

Cost Matrix:
[[0.         0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.         0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.         0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.         0.29122914]
 [0.61185289 0.13949386 0.29214465 0.36636184 0.        ]]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kanaoka/usr/github/caredx2024_rapid_prototyping/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d5a98eed7cb84e60a5a50facf06b6fed-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/d5a98eed7cb84e60a5a50facf06b6fed-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 191 RHS
At line 201 BOUNDS
At line 242 ENDATA
Problem MODEL has 9 rows, 40 columns and 56 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOpt

In [25]:
prob

mTSP:
MINIMIZE
0.9507143064099162*x_0_1_0 + 0.9507143064099162*x_0_1_1 + 0.7319939418114051*x_0_2_0 + 0.7319939418114051*x_0_2_1 + 0.5986584841970366*x_0_3_0 + 0.5986584841970366*x_0_3_1 + 0.15601864044243652*x_0_4_0 + 0.15601864044243652*x_0_4_1 + 0.15599452033620265*x_1_0_0 + 0.15599452033620265*x_1_0_1 + 0.8661761457749352*x_1_2_0 + 0.8661761457749352*x_1_2_1 + 0.6011150117432088*x_1_3_0 + 0.6011150117432088*x_1_3_1 + 0.7080725777960455*x_1_4_0 + 0.7080725777960455*x_1_4_1 + 0.020584494295802447*x_2_0_0 + 0.020584494295802447*x_2_0_1 + 0.9699098521619943*x_2_1_0 + 0.9699098521619943*x_2_1_1 + 0.21233911067827616*x_2_3_0 + 0.21233911067827616*x_2_3_1 + 0.18182496720710062*x_2_4_0 + 0.18182496720710062*x_2_4_1 + 0.18340450985343382*x_3_0_0 + 0.18340450985343382*x_3_0_1 + 0.3042422429595377*x_3_1_0 + 0.3042422429595377*x_3_1_1 + 0.5247564316322378*x_3_2_0 + 0.5247564316322378*x_3_2_1 + 0.2912291401980419*x_3_4_0 + 0.2912291401980419*x_3_4_1 + 0.6118528947223795*x_4_0_0 + 0.611852894722